In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer    # Libreria para hacer el analisis de sentimiento
import gzip     # Libreria para cargar archivos comprimidos
import json     # Libreria para manejar archivos json
import ast      # Libreria para manejar datos con sintaxis abstractas
import pandas as pd     #Libreria para Manerar dataframes


#### 1. Carga de Datos desde los achivos json

In [52]:
data= []    # Inicializa la variable tipo lista
file = gzip.open('data/steam_games.json.gz', 'r')   # Abre el archivo especificado en la ruta y lo carga en file
for linea in file:      # Recorre el archivo linea por linea
    data.append(json.loads(linea))      # Transforma cada linea a formato json y lo agraga a la lista
df_steam_games= pd.DataFrame(data)      # Se crea un DataFrame con el contenido de la lista data


In [53]:
data= []    # Inicializa la variable tipo lista
file = gzip.open('data/user_reviews.json.gz', 'r')      # Abre el archivo especificado en la ruta y lo carga en file
for linea in file:      # Recorre el archivo linea por linea
    data.append(ast.literal_eval(linea.decode('utf-8')))    # Transforma cada linea a formato json y lo agraga a la lista
df_user_reviews= pd.DataFrame(data)     # Se crea un DataFrame con el contenido de la lista data

In [3]:
data= []    # Inicializa la variable tipo lista
file = gzip.open('data/users_items.json.gz', 'r')       # Abre el archivo especificado en la ruta y lo carga en file
for linea in file:    # Recorre el archivo linea por linea
    data.append(ast.literal_eval(linea.decode('utf-8')))    # Transforma cada linea a formato json y lo agraga a la lista
df_user_items= pd.DataFrame(data)       # Se crea un DataFrame con el contenido de la lista data

#### 2. Borrar datos nulos y columas inncesarias de los DataSets. Tranformar columnas Anidadas

- **DataFrame: "df_steam_games"**

In [54]:
df_steam_games = df_steam_games.explode('genres') # se crean filas con los valores aninados de genres

In [55]:
# Tranforma la columna de fechas en string en un valor tipo datetime
df_steam_games['release_date']= pd.to_datetime(df_steam_games['release_date'], errors='coerce')
# Crea una columna con el año en base la fecha de lanzamiento
df_steam_games['release_year']= df_steam_games['release_date'].dt.year
# Se importan los valores nulos del año de lanzamiendo por un 0
df_steam_games.fillna({'release_year': 0}, inplace = True)
# Se cambia el el año a tipo entero
df_steam_games['release_year']= df_steam_games['release_year'].astype(int)

In [56]:
df_steam_games= df_steam_games.dropna(how='all') # Elimina todas las finas que continen todos los valores nulos
df_steam_games= df_steam_games.dropna(subset=['app_name']) # Elimina filas con app_name nulo
df_steam_games= df_steam_games.dropna(subset=['id']) # Elimina filas con id nulos
df_steam_games= df_steam_games.drop(['title','release_date','url','reviews_url','tags', 'specs', 'early_access', 'publisher', 'developer', 'price'], axis=1) # Eliminar columnas innecesarias
df_steam_games= df_steam_games.rename(columns={'app_name': 'game_name'}) # cambiar normbre de la columna
df_steam_games= df_steam_games.rename(columns={'id': 'game_id'}) # cambiar normbre de la columna

In [57]:
df_steam_games.isna().sum()

genres          3282
game_name          0
game_id            0
release_year       0
dtype: int64

In [58]:
df_steam_games.head(5)

,genres,game_name,game_id,release_year
88310,Action,Lost Summoner Kitty,761140,2018
88310,Casual,Lost Summoner Kitty,761140,2018
88310,Indie,Lost Summoner Kitty,761140,2018
88310,Simulation,Lost Summoner Kitty,761140,2018
88310,Strategy,Lost Summoner Kitty,761140,2018


- **DataFrame: "df_user_reviews"**

In [59]:
lista_datos = []    # Se inicializa una lista para guadar los elementos de la columna reviews
id_usuarios = []    # Se inicializa una lista de se guadran los id de usuario para cada fila
# Iterar sobre los registros del DataFrame
for index, fila in df_user_reviews.iterrows():
    for elemento in fila['reviews']:                # Se itera sobre las elementos de la columna reviews
        id_usuarios.append(fila['user_id'])         # Se agreda el id de usuario a la lista
        elem_review = ast.literal_eval(str(elemento))   # Se transforma  a diccionario de cada elemento de la cplumna reviews
        lista_datos.append(elem_review)     # Se agrega a la lista el elemento transformado

In [60]:
df_user_reviews = pd.DataFrame(lista_datos)
df_user_reviews['user_id']= id_usuarios

In [61]:
#Funcion para calcular el analisis de sentimientos

analizador = SentimentIntensityAnalyzer()   # Se instancia un objeto con la clase 
def analisis_sentimiento(review):           
    puntuacion = analizador.polarity_scores(review)     # Calcula la puntuacion del review
    if puntuacion['compound'] > 0.5:    # Si se cumple la condicion se returna 2 (positivo)
        return 2
    if (puntuacion['compound'] < 0.5) & (puntuacion['compound'] >= 0):  # Si se cumple la condicion se returna 1 (Neutro)
        return 1
    else:
        return 0        # Si no se cumpla las condiciones anteriores se retorna 0 (negatrivo)

In [62]:
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analisis_sentimiento) # Se crea un columna con el analisis de sentimmientos calculados

In [63]:
df_user_reviews['sentiment_analysis'].value_counts() # Contar los valores de analisis de sentimientos

sentiment_analysis
2    31581
1    19201
0     8523
Name: count, dtype: int64

In [64]:
df_user_reviews['date'] = pd.to_datetime((df_user_reviews['posted']), format='Posted %B %d, %Y.', errors='coerce') # Crear una fila fecha con tranformacion de posted un fomrato fecha
df_user_reviews['year'] = df_user_reviews['date'].dt.year   # Se crea una columna con el año de la fecha
df_user_reviews.fillna({'year': 0}, inplace = True)     # Se imputan los valores nulos de año a 0
df_user_reviews['year'] = df_user_reviews['year'].astype(int)       # Se cambia la columna año a tipo entero

In [65]:
df_user_reviews = df_user_reviews.rename(columns={'item_id': 'game_id'}) # cambiar normbre de la columna
df_user_reviews = df_user_reviews[['user_id','year', 'game_id', 'recommend', 'sentiment_analysis']] # Se seleccionan solo las culumnas necesarias
df_user_reviews.head()

,user_id,year,game_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,js41637,2014,251610,True,2
4,js41637,2013,227300,True,2


In [66]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   year                59305 non-null  int32 
 2   game_id             59305 non-null  object
 3   recommend           59305 non-null  bool  
 4   sentiment_analysis  59305 non-null  int64 
dtypes: bool(1), int32(1), int64(1), object(2)
memory usage: 1.6+ MB


- **DataFrame: "df_user_items"**

In [4]:
lista_datos = []    # Se inicializa una lista para guadar los elementos de la columna reviews
id_usuarios = []    # Se inicializa una lista de se guadran los id de usuario para cada fila
# Iterar sobre los registros del DataFrame
for index, fila in df_user_items.iterrows(): 
    for elemento in fila['items']:                  # Se itera sobre las elementos de la columna reviews
        id_usuarios.append(fila['user_id'])         # Se agreda el id de usuario a la lista
        item = ast.literal_eval(str(elemento))      # Se transforma  a diccionario de cada elemento de la cplumna reviews
        lista_datos.append(item)        # Se agrega a la lista el elemento transformado

In [5]:
df_user_items = pd.DataFrame(lista_datos)   # se sobre escribe el dataFrame nuevmamentes con la lista de datos anidadas
df_user_items['user_id']= id_usuarios       # Se agrega la columna user_id

In [6]:
df_user_items= df_user_items.rename(columns={'item_name': 'game_name'}) # cambiar normbre de la columna
df_user_items= df_user_items.rename(columns={'item_id': 'game_id'}) # cambiar normbre de la columna
df_user_items['game_id']= df_user_items['game_id'].astype(int)      # Se cambia el tipo de la columna id a entero
df_user_items = df_user_items[['user_id','game_id', 'game_name', 'playtime_forever']] ## Se seleccionan solo las culumnas necesarias

In [7]:
df_user_items.drop_duplicates(inplace=True)     # Elimina filas duplicadas

In [8]:
df_user_items.head()

,user_id,game_id,game_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


In [9]:
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5094092 entries, 0 to 5153208
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   game_id           int32 
 2   game_name         object
 3   playtime_forever  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 174.9+ MB


#### 3. Exportar los Dataframes transformados a archivos CSV y parquet

In [67]:
df_steam_games.to_csv('DataSet_tranformados/games.csv', index=False)    #Exportar df_steam_games a un archivo games.cvs

In [68]:
df_user_reviews.to_csv('DataSet_tranformados/reviews.csv', index=False) #Exportar df_user_reviews a un archivo reviews.cvs

In [10]:
df_user_items.to_parquet('DataSet_tranformados/items.parquet', index=False) #Exportar df_user_reviews a un archivo items.parquet

In [11]:
#df_user_items.iloc[1,-1]
#user_reviews_temp = user_reviews.explode('reviews')
#user_reviews_temp.head(10)